# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    #TODO 1: Add your code here
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    print(len(df_eng))
    print(len(df_fr))
    print(len(df_sp))
    df_eng.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_fr.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df_sp.columns = ['Title', 'Year', 'Synopsis', 'Review']
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    return df

df = preprocess_data()

10
10
10


In [3]:
df.sample(10)

,Title,Year,Synopsis,Review
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s..."
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale..."
14,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi..."
17,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues..."
28,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu..."
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage..."
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of..."
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su..."
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte..."
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi..."


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
# TODO 2: Update the code below
# Load model directly

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return decoded

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/home/swaggyprophet/.local/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

In [5]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df.loc[10:19, 'Review'].tolist()
fr_reviews_en = [translate(review, fr_en_model, fr_en_tokenizer) for review in fr_reviews]

# filter synopsis in French and translate to English
fr_synopsis = df.loc[10:19, 'Synopsis'].tolist()
fr_synopsis_en = [translate(synopsis, fr_en_model, fr_en_tokenizer) for synopsis in fr_synopsis]

# filter reviews in Spanish and translate to English
es_reviews = df.loc[20:29, 'Review'].tolist()
es_reviews_en = [translate(review, es_en_model, es_en_tokenizer) for review in es_reviews]

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[20:29, 'Synopsis'].tolist()
es_synopsis_en = [translate(synopsis, es_en_model, es_en_tokenizer) for synopsis in es_synopsis]

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[10:19, 'Review'] = fr_reviews_en
df.loc[10:19, 'Synopsis'] = fr_synopsis_en
df.loc[20:29, 'Review'] = es_reviews_en
df.loc[20:29, 'Synopsis'] = es_synopsis_en

In [6]:
df.sample(10)

,Title,Year,Synopsis,Review
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ..."
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ..."
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi..."
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with..."
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina..."
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e..."
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of..."
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie..."
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ..."
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi..."


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "Seethal/sentiment_analysis_generic_dataset"
sentiment_classifier = pipeline("text-classification", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    sentiment = classifier(text)
    if sentiment[0]['label'] == 'LABEL_0':
        return 'Negative'
    elif sentiment[0]['label'] == 'LABEL_1':
        return 'Neutral'
    elif sentiment[0]['label'] == 'LABEL_2':
        return 'Positive'
    else:
        return 'Sentiment not recognized'

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
# Appears that LABEL_2 = positive, LABEL_1 = nuetral, and LABEL_0 = negative
df['Sentiment'] = df['Review'].apply(lambda review: analyze_sentiment(review, sentiment_classifier))

In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Sentiment
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",Neutral
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Positive
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Negative
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",Negative
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",Positive
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Negative
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",Positive
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",Positive
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",Positive
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",Negative


In [11]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)